In [1]:
from gurobipy import Model, quicksum, GRB

In [2]:
Cust = set(range(1,3))
DC = Cust.copy() 
Route = {(i,j) for i in DC for j in Cust}
CT = {1,2}

T = 3
Period = list(range(1, T+1))

In [3]:
transport = {i:1 for i in DC}
delivery = {(i,j):5 for i,j in Route}
dc_run = {(i,ct):100*ct for i in DC for ct in CT}
dc_stock = {i:1 for i in DC}

demand = {(j,t):10 for j in Cust for t in Period}

In [4]:
model = Model()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [5]:
X, x, y, sD = {}, {}, {}, {}

for i in DC:
    for t in Period:
        X[i,t] = model.addVar(vtype="C", name=f'X[{i},{t}]')

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for i in DC:
    for ct in CT:
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

for i in DC:
    sD[i,0] = model.addVar(vtype="C", ub=0, name=f'sD[{i},{t}]')
    for t in Period:
        sD[i,t] = model.addVar(vtype="C", name=f'sD[{i},{t}]')

model.update()

In [6]:
Cust_Demand_Cons, DC_Running_Cons, CT_Only_Cons, DC_Flow_Cons = {}, {}, {}, {}

for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC)
            ==
            demand[j,t]
        )

for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t_] for t_ in Period) * quicksum(y[i,ct,t] for ct in CT)
        )

for i in DC:
    for t in Period:
        CT_Only_Cons[i,t] = model.addConstr(
            quicksum(y[i,ct,t] for ct in CT)
            <=
            1
        )

for i in DC:
    for t in Period:
        DC_Flow_Cons = model.addConstr(
            X[i,t] + sD[i,t-1]
            ==
            quicksum(x[i,j,t] for j in Cust) + sD[i,t]
        )
    
model.update()

In [7]:
model.setObjective(
    quicksum(transport[i] * X[i,t] for i in DC for t in Period) +
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +
    quicksum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(dc_stock[i] * sD[i,t] for i in DC for t in Period)
    ,GRB.MINIMIZE
)

model.update()

In [8]:
model.optimize()

Optimize a model with 30 rows, 38 columns and 90 nonzeros
Variable types: 26 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 80360.000000
Presolve removed 30 rows and 38 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 30360 80360 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.036000000000e+04, best bound 3.036000000000e+04, gap 0.0000%


In [9]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

X[2,1] 20.0
X[2,2] 20.0
X[2,3] 20.0
x[2,1,1] 10.0
x[2,1,2] 10.0
x[2,1,3] 10.0
x[2,2,1] 10.0
x[2,2,2] 10.0
x[2,2,3] 10.0
y[2,1,1] 1.0
y[2,1,2] 1.0
y[2,1,3] 1.0
